In [1]:
import os
import s3fs
import pandas as pd

os.environ["AWS_ACCESS_KEY_ID"] = 'PVZT77T3X1860L6FCJFG'
os.environ["AWS_SECRET_ACCESS_KEY"] = '8Uq1Hnf5y5f5K47efWXkZJ6kdQiyOy9IBa5D9MnN'
os.environ["AWS_SESSION_TOKEN"] = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJQVlpUNzdUM1gxODYwTDZGQ0pGRyIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzY5NjEyMTg2LCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6Imx1Y2FzLmN1bXVuZWxAZW5zYWUuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzcxMzE1NTA1LCJmYW1pbHlfbmFtZSI6IkN1bXVuZWwiLCJnaXZlbl9uYW1lIjoiTHVjYXMiLCJncm91cHMiOlsiVVNFUl9PTllYSUEiLCJzdGF0YXBwLXNlZ21lZGljIl0sImlhdCI6MTc3MDcxMDcwNSwiaXNzIjoiaHR0cHM6Ly9hdXRoLmxhYi5zc3BjbG91ZC5mci9hdXRoL3JlYWxtcy9zc3BjbG91ZCIsImp0aSI6Im9ucnRydDo1ZjM1MWE3MS03NTIwLWYyNjYtNjBjOS1iYWY5NjYzMTEyYjciLCJuYW1lIjoiTHVjYXMgQ3VtdW5lbCIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJsYWIiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtc3NwY2xvdWQiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdLCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGdyb3VwcyBlbWFpbCIsInNpZCI6Ijc4ODAyYmFkLTFjZTEtODFkOC1mYjI4LTQwNWY4MWQ0ZGVmNSIsInN1YiI6ImUyZDc4NjRjLTcwMzItNDI0ZC04OTA2LWU0ZjhiNDFjYzAwMyIsInR5cCI6IkJlYXJlciJ9.Dlw2Tbo6kDKyKT70uHQjFhqBaOTtzYXw5t9pKQu7I-GxTbbL_qo7AMAHIWUfsPj-l4JDZUf6RabEsg-Rx3A39g'
os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])

with fs.open("s3://lab/art_net_dec.parquet") as f:
    jdf = pd.read_parquet(f)

In [43]:
import pandas as pd
import numpy as np
import json

def build_author_coauthor(df, authors_col="authors"):
    rows = []

    for _, row in df.iterrows():
        authors = row[authors_col]

        # convertir JSON string
        if isinstance(authors, str):
            try:
                authors = json.loads(authors)
            except:
                continue

        # convertir numpy array -> list
        if isinstance(authors, np.ndarray):
            authors = authors.tolist()

        if not isinstance(authors, list):
            continue

        names = [
            a.get("name")
            for a in authors
            if isinstance(a, dict) and a.get("name")
        ]

        for author in names:
            coauthors = [a for a in names if a != author]

            rows.append({
                "author": author,
                "coauthors": coauthors,
                "year" : row["year"]
            })

    return pd.DataFrame(rows)


In [44]:
df = build_author_coauthor(jdf)

In [46]:
#more than 10 pub
df = df[df["author"].map(df["author"].value_counts()) > 10]

In [49]:
#last pub
last = df.groupby("author")["year"].max().reset_index()


In [57]:
import pandas as pd

def dynamic_inactive_2hop_fast(df, inactivity_window=5):
    """
    For each author and year, compute the number of 2-hop coauthors that are inactive.
    
    Parameters:
    - df: DataFrame with columns ['author', 'coauthors', 'year']
          - 'coauthors' must be a list of author names
    - inactivity_window: number of years to consider someone inactive
    
    Returns:
    - DataFrame: columns ['author', 'year', 'inactive_coauthors_2hop', 'total_2hop', 'inactive_ratio']
    """

    # -------------------------
    # Step 0: Ensure numeric years
    # -------------------------
    df = df.copy()
    df["year"] = pd.to_numeric(df["year"], errors="coerce")
    df = df.dropna(subset=["year"])
    df["year"] = df["year"].astype(int)

    # -------------------------
    # Step 1: Compute last publication per author
    # -------------------------
    last_year = df.groupby("author")["year"].max().to_dict()

    # -------------------------
    # Step 2: Prepare graph as dict-of-sets
    # -------------------------
    neighbors = dict()  # author -> set of direct coauthors

    # -------------------------
    # Step 3: Loop through years incrementally
    # -------------------------
    results = []
    years = sorted(df["year"].unique())

    for Y in years:
        # Current year publications
        df_current = df[df["year"] == Y]

        # Update graph with new edges
        for row in df_current.itertuples(index=False):
            author = row.author
            coauthors = row.coauthors
            if not isinstance(coauthors, list):
                continue

            if author not in neighbors:
                neighbors[author] = set()

            for c in coauthors:
                if c not in neighbors:
                    neighbors[c] = set()
                neighbors[author].add(c)
                neighbors[c].add(author)

        # Determine inactive authors at year Y
        inactive = {a for a, ly in last_year.items() if ly <= Y - inactivity_window}

        # -------------------------
        # Step 4: Count 2-hop inactive neighbors
        # -------------------------
        for author in neighbors:
            co1 = neighbors[author]
            co2 = set()
            for c in co1:
                co2.update(neighbors[c])
            co2.discard(author)
            co2 -= co1

            inactive_count = len(co2 & inactive)
            total_2hop = len(co2)
            ratio = inactive_count / total_2hop if total_2hop > 0 else 0

            results.append({
                "author": author,
                "year": Y,
                "inactive_coauthors_2hop": inactive_count,
                "total_2hop": total_2hop,
                "inactive_ratio": ratio
            })
            print(Y)

    return pd.DataFrame(results)


In [ ]:
result = dynamic_inactive_2hop_fast(df)